# Coding a RAPTOR toy example

## Goal

Learn the RAPTOR algorithm by coding it for a toy example with the data structures advised in the paper. We code RAPTOR for a super simple toy example with only two routes and two trips each.

## Toy example
- TODO updates:
    - additional route r2 that goes from A to E slowly
    - walking paths
![toy_example](img/RAPTOR_example.png) 

## Encoding the data structures
### General considerations
We adhere to the data structures proposed by Delling et al. These structures aim to minimize read times in memory by making use of consecutive in-memory adresses. Thus, structures with varying dimensions (e.g dataframes, python lists) are excluded. We illustrate the difficulty with an example. 

Each route has a potentially unique number of stops. Therefore, we cannot store stops in a 2D array of routes by stops, as the number of stops is not the same for each route. We adress this problem by storing stops consecutively by route, and keeping track of the index of the first stop for each route.

This general strategy is applied to all the required data structures.

### routes
The `routes` array will contain arrays `[n_trips, n_stops, pt_1st_stop, pt_1st_trip]` where all four values are `int`. To avoid overcomplicating things and try to mimic pointers in python, `pt_1st_stop` and `pt_1st_trip` contain integer indices.

In [35]:
import numpy as np
routes = np.array([[2, 3, 0, 0], #r0
                   [2, 3, 3, 6], #r1
                  [2, 2, 6, 12]]) # r2

### routeStops
`routeStops` is an array that contains the ordered lists of stops for each route. `pt_1st_stop` in `routes` is required to get to the first stop of the route. is itself an array that contains the sequence of stops for route $r_i$.

In [36]:
routeStops = np.array([0, 1, 2, # A, B, C
                       3, 2, 4, # D, C, E
                      0, 4]) # A, E

### stopTimes

The i-th entry in the `stopTimes` array is itself an array which contains the arrival and departure time at a particular stop for a particular trip. `stopTimes` is sorted by routes, and then by trips. We retrieve the index of the first (earliest) trip of the route with the pointer `pt_1st_trip` stored in `routes`. We may use the built-in `numpy` [date and time data structures](https://blog.finxter.com/how-to-work-with-dates-and-times-in-python/). In short, declaring dates and times is done like this: `np.datetime64('YYYY-MM-DDThh:mm')`. Entries with a `NaT` arrival or departure times correspond to beginning and end of trips respectively.

Note that trips are indexed implicitely in stopTimes, but we decided to change a little bit from the paper and index them according to their parent route instead of giving them an absolute index. It makes things a bit easier when coding the algorithm.

In [37]:
stopTimes = np.array([
    # r0, t0
    [None, '2020-05-11T08:00'],
    ['2020-05-11T08:25', '2020-05-11T08:30'],
    ['2020-05-11T08:55', None],

    # ro, t1
    [None, '2020-05-11T08:10'],
    ['2020-05-11T08:35', '2020-05-11T08:40'],
    ['2020-05-11T09:05', None],
    
    # r1, t0 
    [None, '2020-05-11T08:00'],
    ['2020-05-11T08:05', '2020-05-11T08:10'],
    ['2020-05-11T08:15', None],

    # r1, t1
    [None, '2020-05-11T09:00'],
    ['2020-05-11T09:05', '2020-05-11T09:10'],
    ['2020-05-11T09:15', None],
    
    #r2, t0
    [None, '2020-05-11T08:20'],
    ['2020-05-11T09:20', None],
    
    #r2, t1
    [None, '2020-05-11T08:30'],
    ['2020-05-11T09:30', None]],
    dtype='datetime64')

`NaT` is the `None` equivalent for `numpy datetime64`.

In [38]:
np.isnat(stopTimes[0])

array([ True, False])

### stopRoutes

`stopRoutes` contains the routes associated with each stop. We need the pointer in `stops` to index `stopRoutes` correctly.

In [39]:
stopRoutes = np.array([0, 2, # A
                       0, # B
                       0,1, # C
                       1, # D
                       1, 2]) # E

We should also build an array for transfer times (including walking times), but for now let's ignore this additional complexity. Finally, the i-th entry in the `stops` array points to the first entry in `stopRoutes` (and `transfers` when that will be tried) associated with stop $p_i$

In [40]:
stops = np.array([[0, None],# A
                 [2, None], # B
                 [3, None],# C
                 [5, None], # D
                 [6, None], # E
                 [len(stopRoutes), None]]) # fictive stop to account for length of E

## Coding the standard RAPTOR

Below, we code RAPTOR as it is described in the paper, with all optimizations. That corresponds to the pseudocode block in the article. It solves the earliest arrival time problem: we enter an start stop, a target stop and a departure time and it finds the earliest arrival time in k rounds (i.e taking at most k transports). Note that walking between stops is not considered a transport.

In [41]:
p_s = 0 # start stop = A
p_t = 4 # target stop = E
tau_0 = np.datetime64('2020-05-11T08:05') # departure time 08:05
k_max = 10 # we set a maximum number of transports to pre-allocate memory for the numpy array tau_i

In [42]:
def raptor_standard(p_s, p_t, tau_0, routes, routeStops, stopTimes, stopRoutes, stops,
                    k_max=10):
    
    #******************************************initialization******************************************
    n_stops = len(stops)-1 # to remove the fictive stop to account for all the routes belonging to the last stop

    # earliest arrival time at each stop for each round.
    tau = np.full(shape=(k_max, n_stops), fill_value = np.datetime64('2100-01-01T00:00')) # 2100 instead of infinity # number of stops * max number of transports

    # earliest arrival time at each stop, indep. of round
    tau_star = np.full(shape=n_stops, fill_value = np.datetime64('2100-01-01T00:00'))

    # to backtrack the journey of TRANSPORTS once it is finished
    #[route, trip, boarding stop, exit stop]
    # we will keep [r, t, p_b, p_e, p_f1, pf2, t_w] i.e 
            # [route, trip (offset by route, not absolute), boarding stop, exit stop, beginning stop of the walk, target stop of the walk, time walked]
    journey = np.full(shape=(k_max, n_stops, 7), fill_value = -1, dtype=int)
    
    marked = [p_s]
    q = []
    tau[0, p_s] = tau_0
    
    #Maybe TODO (but not in original raptor): footpaths from the departure stop

    #****************************************** main loop******************************************
    for k in np.arange(1, k_max+1):
        print('\n******************************STARTING round k={}******************************'.format(k))
        # accumulate routes serving marked stops from previous rounds
        q = []
        marked = list(set(marked)) # removing potential duplicate stops in marked due to walking paths
        print('Marked stops at the start of the round: {}'.format(marked))
        for p in marked:
            for r in stopRoutes[stops[p][0]:stops[p+1][0]]: # foreach route r serving p
                print('Route considered for the queue: ({0}, {1})'.format(r, p))
                inQueue = False
                for idx, (rPrime, pPrime) in enumerate(q): 
                    # is there already another stop from the same route in q ?
                    if (rPrime == r): 
                        # is there already a later stop from the same route in q ?
                        if(np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == pPrime)[0][0] >\
                           np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p)[0][0]):
                            #in that case, replace the later stop pPrime by stop p in q
                            q[idx] = (r, p)
                            inQueue = True
                            # is there already an earlier stop from the same route in q ?
                        else:
                            # in that case, do not add p to the q.
                            inQueue=True
                if not inQueue:
                    q.append((r, p))

        marked = [] # unmarking all stops

        print('Queue before traversing each route: {}'.format(q))
        # traverse each route
        for (r, p) in q:
            print('\n****TRAVERSING ROUTE r={0} from stop p={1}****'.format(r, p))
            # t is the t-th trip in route r, not the t-th trip in all trips. This makes things easier
            t = None
            # we will keep [r, t, p_b, p_e, p_f, t_w] i.e 
            # [route, trip (offset by route, not absolute), boarding stop, exit stop, target stop of the walk, time walked]
            t_journey = np.empty(4, dtype=int)# contains tripID, board and exit stops to backtrack the journey


            # we only traverse the route starting at p, not from the beginning of the route
            for p_i in routeStops[routes[r][2]+np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p)[0][0]:\
                                   routes[r][2]+routes[r][1]]:
                print("p_i: {}".format(p_i))

                if (t is not None):
                    # 1st trip of route + 
                    # offset for the right trip + 
                    # offset for the right stop
                    arr_t_p_i = stopTimes[routes[r][3] + \
                              t * routes[r][1] + \
                              np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i)[0][0]][0]
                    print("arr_t_p_i: {}".format(arr_t_p_i))

                    if arr_t_p_i < min(tau_star[p_i], tau_star[p_t]):
                        tau[k][p_i] = arr_t_p_i
                        tau_star[p_i] = arr_t_p_i
                        marked.append(p_i)
                        # keep a trace that we went down the trip taken before at this stop
                        t_journey[3] = p_i
                        journey[k][p_i][0:4] = t_journey
                # Can we catch an earlier trip at p_i ?
                print('\n----scanning departure times for route r={0} at stop p_i={1}----'.format(r, p_i))
                t_r = 0
                while True:
                    t_r_dep = stopTimes[routes[r][3]+\
                         # offset corresponding to stop p_i in route r
                         np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i)[0][0] + \
                         routes[r][1]*t_r][1]

                    print("Earliest arrival time at previous step: tau[k-1][p_i]: {}".format(tau[k-1][p_i]))
                    print("Departure time considered: t_r_dep: {}".format(t_r_dep))
                    # We hop on the first trip that departs later than our arrival time at p_i in k-1 transports
                    if t_r_dep > tau[k-1][p_i]:
                        t = t_r
                        print('\n!!!!Hopped on route r={0}, trip t={1} at stop p_i={2}!!!!'.format(r, t, p_i))

                        # here we probably need to save the trip and boarding stop (boarding time will not be useful)
                        t_journey[0] = r
                        t_journey[1] = t
                        t_journey[2] = p_i
                        break
                    t_r += 1

                    # we could not hop on any trip at this stop
                    if t_r == routes[r][0]:
                        break
                        
        print('\n****FOOTPATHS****')
        
        marked_footpaths = [] # storing marked stops for footpaths in a separate list to avoid inifinite loops
        for p in marked:
            if stops[p][1] is not None:
                print('checking walking paths from stop {}'.format(p))
                # making sure there are footpaths for that stop
                # finding the next stop where there are footpaths to find the next index
                next_stop = p
                next_stop_found = False
                while next_stop < len(stops)-1: #carefully check that's the correct version
                    next_stop = next_stop+1
                    if stops[next_stop][1] is not None:
                        next_stop_found = True
                        break
                
                # reinitializing next_stop to p in case no next stop with not 'None' stops[p][1] is found
                if not next_stop_found:
                    next_stop = p+1 # this works because transfers[p:None] is equivalent to transfers[p:]
                    
                        
                for f in transfers[stops[p][1]:stops[next_stop][1]]:
                    print("Considering footpaths from {} to {}".format(p, f[0]))
                    
                    # we only consider footpaths if they strictly ameliorate the arrival time at the arrival stop of the path.
                    if(tau[k][p]+np.timedelta64(f[1], 's') < min(tau_star[f[0]], tau_star[p_t])): 
                        print("Walking to {} is faster !".format(f[0]))
                        tau[k][f[0]] = tau[k][p]+np.timedelta64(f[1], 's')
                        tau_star[f[0]] = tau[k][p]+np.timedelta64(f[1], 's')
                        marked_footpaths.append(f[0])
                        
                        # keeping tracks of footpaths to backtrack the journey:
                        # [departure stop, arrival stop, walking time]
                        journey[k][f[0]][4:7] = [p, f[0], f[1]]
        
        marked.extend(marked_footpaths) # to avoid infinite loops if marked gets appended dynamically
        # stopping criterion: no stops were marked
        if not marked:
            break
    return(tau, tau_star, k, journey)

In [43]:
tau, tau_star, k, journey = raptor_standard(p_s, p_t, tau_0, 
                                            routes = routes, routeStops = routeStops, stopTimes = stopTimes, stopRoutes = stopRoutes, stops = stops)


******************************STARTING round k=1******************************
Marked stops at the start of the round: [0]
Route considered for the queue: (0, 0)
Route considered for the queue: (2, 0)
Queue before traversing each route: [(0, 0), (2, 0)]

****TRAVERSING ROUTE r=0 from stop p=0****
p_i: 0

----scanning departure times for route r=0 at stop p_i=0----
Earliest arrival time at previous step: tau[k-1][p_i]: 2020-05-11T08:05
Departure time considered: t_r_dep: 2020-05-11T08:00
Earliest arrival time at previous step: tau[k-1][p_i]: 2020-05-11T08:05
Departure time considered: t_r_dep: 2020-05-11T08:10

!!!!Hopped on route r=0, trip t=1 at stop p_i=0!!!!
p_i: 1
arr_t_p_i: 2020-05-11T08:35

----scanning departure times for route r=0 at stop p_i=1----
Earliest arrival time at previous step: tau[k-1][p_i]: 2100-01-01T00:00
Departure time considered: t_r_dep: 2020-05-11T08:30
Earliest arrival time at previous step: tau[k-1][p_i]: 2100-01-01T00:00
Departure time considered: t_r_dep:

In [44]:
tau_star

array(['2100-01-01T00:00', '2020-05-11T08:35', '2020-05-11T09:05',
       '2100-01-01T00:00', '2020-05-11T09:15'], dtype='datetime64[m]')

In [45]:
k_last = k

tau[0:k_last+1]

array([['2020-05-11T08:05', '2100-01-01T00:00', '2100-01-01T00:00',
        '2100-01-01T00:00', '2100-01-01T00:00'],
       ['2100-01-01T00:00', '2020-05-11T08:35', '2020-05-11T09:05',
        '2100-01-01T00:00', '2020-05-11T09:20'],
       ['2100-01-01T00:00', '2100-01-01T00:00', '2100-01-01T00:00',
        '2100-01-01T00:00', '2020-05-11T09:15'],
       ['2100-01-01T00:00', '2100-01-01T00:00', '2100-01-01T00:00',
        '2100-01-01T00:00', '2100-01-01T00:00']], dtype='datetime64[m]')

`journey` contains all the necessary information to backtrack from the solution to the actual journey in terms of sequence of transports.

`journey` has dimensions `k` by `n_stops` by 4+3.
- The 4 first values store the route and trip taken, the departure and arrival stops.
- The 3 last values are used by footpaths. They contain the departure stop for the walk, the arrival stop for the walk and the walking time in seconds.

When we hop on a trip, we store the trip (with the route) and the boarding and exit stops as the array `t_journey`: `(r, t, p_boarding, p_exit)`. At each stop `p_i` where we ameliorate the arrival time in round `k`, we store `t_journey` in the first 4 cells of `journey[k][p_i]`. `p_i` corresponds to the exit stop when backtracking.

When walking to stop `p_i` is shorter, we store the departure, arrival stops and walking time in the 3 last cells of `journey[k][p_i]`.

The end result is a `journey` array which contains -1 values in all seven cells in `journey[k][p_i]` if the arrival time at `p_i` was not ameliorated at step `k`. `journey[k][p_i]` where there are values other than -1 indicate that the arrival time was ameliorated either by walking or by taking a transport. 

In [46]:
journey[0:k_last]

array([[[-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1]],

       [[-1, -1, -1, -1, -1, -1, -1],
        [ 0,  1,  0,  1, -1, -1, -1],
        [ 0,  1,  0,  2, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [ 2,  0,  0,  4, -1, -1, -1]],

       [[-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1],
        [ 1,  1,  2,  4, -1, -1, -1]]])

## Backtracking

We reconstruct the actual journey from the `journey` array by backtracking from the arrival stop to the departure stop. At each round k where we notice that the arrival time for the target stop was ameliorated, we start a new leg corresponding to a journey reaching the target stop in k transports.

When backtracking without footpaths, it is sufficient at each round k to check at which stop the trip at round k-1 began. 



In [47]:
def backtrack_journey(k_last, p_t, journey):
    # journey_act = actual journey, will contain the sequence of transports in the correct order
    journey_act = [[] for k in range(0, k_last)] # there's maximum k routes to get to the final stop
    p_board = p_t
    n_legs = 1 # each leg is a journey to from the departure stop to the target stop in exactly k transports
    journey_found = False

    # iterating backwards in rounds from k_last -1 to 1
    for k in range(k_last-1, 0, -1): # second argument in range is not included in the boundaries
        # Was the tarrival time at the target stop ameliorated at round k ? 
        if np.any(journey[k][p_t]!=np.array([-1, -1, -1, -1, -1, -1, -1])):

            # starting a new leg in the list of actual journeys
            journey_found = True
            # iterating from k to 0 to reconstruct the actual journey in k transports
            p_board = p_t
            for k_prime in range(k, 0, -1):

                # did we get to that stop by walking ?
                if journey[k_prime][p_board][5] !=-1:

                    # we keep track of the stop to which we walked to as well as the departure stop of the walk
                    stop_walk_dep = journey[k_prime][p_board][4]
                    journey_act[k].append([journey[k_prime][stop_walk_dep], journey[k_prime][p_board]])
                    p_board = journey[k_prime][stop_walk_dep][2]

                # we did not get to that stop by walking
                else:

                    journey_act[k].append(journey[k_prime][p_board])
                    p_board = journey[k_prime][p_board][2]

    # reversing the order of journey_act to get journeys from the start stop to the target stop
    journey_act = [j[::-1] for j in journey_act]

    # building a human readable output for the trip:
    for k, j in enumerate(journey_act):

        if j: # going only through non-empty journeys
            print('******************JOURNEY IN {} TRIPS******************'.format(k))
            print('raw representation of the journey in {} trips: {}'.format(k, j))

            for k_prime, t in enumerate(j):
                # We did not walk at step k
                if len(t) !=2:
                    p_boarding = t[2]
                    p_exit = t[3]
                    r_k = t[0]
                    time_boarding = stopTimes[routes[r_k][3] + \
                                              np.where(routeStops[routes[r_k][2]:routes[r_k][2]+routes[r_k][1]] == p_boarding)[0][0] + \
                                              t[1]*routes[r_k][1]][1]
                    time_exit = stopTimes[routes[r_k][3] + \
                                              np.where(routeStops[routes[r_k][2]:routes[r_k][2]+routes[r_k][1]] == p_exit)[0][0] + \
                                              t[1]*routes[r_k][1]][0]
                    print("At stop {}, take route {} leaving at time {} \n...".format(p_boarding, r_k, time_boarding))

                    print(" and exit at stop {} at time {}".format(p_exit, time_exit))

                # We walked at step k
                elif len(t)==2:
                    print(t)
                    p_boarding = t[0][2]
                    p_exit = t[0][3]
                    r_k = t[0][0]
                    time_boarding = stopTimes[routes[r_k][3] + \
                                              np.where(routeStops[routes[r_k][2]:routes[r_k][2]+routes[r_k][1]] == p_boarding)[0][0] + \
                                              t[0][1]*routes[r_k][1]][1]
                    time_exit = stopTimes[routes[r_k][3] + \
                                              np.where(routeStops[routes[r_k][2]:routes[r_k][2]+routes[r_k][1]] == p_exit)[0][0] + \
                                              t[0][1]*routes[r_k][1]][0]
                    p_start_walk = t[1][4]
                    p_end_walk = t[1][5]
                    walk_duration = t[1][6]/60

                    print("At stop {}, take route {} leaving at time {} \n...".format(p_boarding, r_k, time_boarding))

                    print("... exit at stop {} at time {}... ".format(p_exit, time_exit))

                    print("and walk for {} minutes from stop {} to stop {}.".format(walk_duration, p_start_walk, p_end_walk))
    
    if not journey_found:
        print('No journey was found for this query')
    return journey_found        

In [48]:
backtrack_journey(k_last, p_t, journey);

******************JOURNEY IN 1 TRIPS******************
raw representation of the journey in 1 trips: [array([ 2,  0,  0,  4, -1, -1, -1])]
At stop 0, take route 2 leaving at time 2020-05-11T08:20 
...
 and exit at stop 4 at time 2020-05-11T09:20
******************JOURNEY IN 2 TRIPS******************
raw representation of the journey in 2 trips: [array([ 0,  1,  0,  2, -1, -1, -1]), array([ 1,  1,  2,  4, -1, -1, -1])]
At stop 0, take route 0 leaving at time 2020-05-11T08:10 
...
 and exit at stop 2 at time 2020-05-11T09:05
At stop 2, take route 1 leaving at time 2020-05-11T09:10 
...
 and exit at stop 4 at time 2020-05-11T09:15


## Let's add footpaths

For now, we have not tried including footpaths in our dataset. Below, we modify the timetable by adding a new route r3, which links a new stop F to E in a single travel. F may be reached in a very long time from A, but in a short time from B, meaning that it should become shorter to:

- Take a trip from A to B
- Walk from B to F
- Take a trip from F to E

rather than the current best trip:
- Take a trip from A to C
- Take a trip from C to E


Note that the single transport solution:
- Take a trip from A to E

should still appear as the optimal solution for k = 1, i.e one transport is taken.

In [49]:
routes = np.array([[2, 3, 0, 0], #r0
                   [2, 3, 3, 6], #r1
                  [2, 2, 6, 12], #r2
                 [2, 2, 8, 16]]) # r3

In [50]:
routeStops = np.array([0, 1, 2, # A, B, C
                       3, 2, 4, # D, C, E
                      0, 4, # A, E
                      5, 4]) #F, E

In [51]:
stopTimes = np.array([
    # r0, t0
    [None, '2020-05-11T08:00'],
    ['2020-05-11T08:25', '2020-05-11T08:30'],
    ['2020-05-11T08:55', None],

    # ro, t1
    [None, '2020-05-11T08:10'],
    ['2020-05-11T08:35', '2020-05-11T08:40'],
    ['2020-05-11T09:05', None],
    
    # r1, t0 
    [None, '2020-05-11T08:00'],
    ['2020-05-11T08:05', '2020-05-11T08:10'],
    ['2020-05-11T08:15', None],

    # r1, t1
    [None, '2020-05-11T09:00'],
    ['2020-05-11T09:05', '2020-05-11T09:10'],
    ['2020-05-11T09:15', None],
    
    #r2, t0
    [None, '2020-05-11T08:20'],
    ['2020-05-11T09:20', None],
    
    #r2, t1
    [None, '2020-05-11T08:30'],
    ['2020-05-11T09:30', None],
    
    #r3, t0
    [None, '2020-05-11T08:05'],
    ['2020-05-11T08:25', None],

    #r3, t1
    [None, '2020-05-11T08:45'],
    ['2020-05-11T09:05', None]],
    dtype='datetime64')

In [52]:
stopRoutes = np.array([0, 2, # A
                       0, # B
                       0,1, # C
                       1, # D
                       1, 2, 3, # E
                       3]) # F

## transfers
The `transfers` is a 2D `np.ndarray` where each entry `[p_j, time]` represents the time it takes to reach p_j from stop p_i. The correspondance between the indexing of `transfers` and p_i is done via `stops[p_i][1]`, i.e the first entry in `transfers` containing a connection from stop p_i.

As we cannot store different data types in numpy arras, `time` will have to be converted to `np.timedelta64`, the format used to make differences between `np.datetime.64` variables. We will consider all `time` values as **positive values in seconds**.

In [53]:
stopTimes[0][1] - stopTimes[1][1]

numpy.timedelta64(-30,'m')

In [54]:
np.timedelta64(30, 's')

numpy.timedelta64(30,'s')

In [55]:
transfers = np.array([[5, 3600], # A -> F
                      [5, 300], # B -> F
                      [0, 3600], # F -> A
                      [1, 300] # F -> A
                     ])

In [56]:
stops = np.array([[0, 0],# A
                 [2, 1], # B
                 [3, None],# C
                 [5, None], # D
                 [6, None], # E
                  [9, 2], # F
                 [len(stopRoutes), None]]) # fictive stop to account for length of E

In [57]:
tau, tau_star, k, journey = raptor_standard(p_s, p_t, tau_0, 
                                            routes = routes, routeStops = routeStops, stopTimes = stopTimes, stopRoutes = stopRoutes, stops = stops)


******************************STARTING round k=1******************************
Marked stops at the start of the round: [0]
Route considered for the queue: (0, 0)
Route considered for the queue: (2, 0)
Queue before traversing each route: [(0, 0), (2, 0)]

****TRAVERSING ROUTE r=0 from stop p=0****
p_i: 0

----scanning departure times for route r=0 at stop p_i=0----
Earliest arrival time at previous step: tau[k-1][p_i]: 2020-05-11T08:05
Departure time considered: t_r_dep: 2020-05-11T08:00
Earliest arrival time at previous step: tau[k-1][p_i]: 2020-05-11T08:05
Departure time considered: t_r_dep: 2020-05-11T08:10

!!!!Hopped on route r=0, trip t=1 at stop p_i=0!!!!
p_i: 1
arr_t_p_i: 2020-05-11T08:35

----scanning departure times for route r=0 at stop p_i=1----
Earliest arrival time at previous step: tau[k-1][p_i]: 2100-01-01T00:00
Departure time considered: t_r_dep: 2020-05-11T08:30
Earliest arrival time at previous step: tau[k-1][p_i]: 2100-01-01T00:00
Departure time considered: t_r_dep:

In [58]:
k_last = k
tau[0:k_last+1]

array([['2020-05-11T08:05', '2100-01-01T00:00', '2100-01-01T00:00',
        '2100-01-01T00:00', '2100-01-01T00:00', '2100-01-01T00:00'],
       ['2100-01-01T00:00', '2020-05-11T08:35', '2020-05-11T09:05',
        '2100-01-01T00:00', '2020-05-11T09:20', '2020-05-11T08:40'],
       ['2100-01-01T00:00', '2100-01-01T00:00', '2100-01-01T00:00',
        '2100-01-01T00:00', '2020-05-11T09:05', '2100-01-01T00:00'],
       ['2100-01-01T00:00', '2100-01-01T00:00', '2100-01-01T00:00',
        '2100-01-01T00:00', '2100-01-01T00:00', '2100-01-01T00:00']],
      dtype='datetime64[m]')

In [59]:
tau_star

array(['2100-01-01T00:00', '2020-05-11T08:35', '2020-05-11T09:05',
       '2100-01-01T00:00', '2020-05-11T09:05', '2020-05-11T08:40'],
      dtype='datetime64[m]')

In [60]:
journey[0:k_last]

array([[[ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1]],

       [[ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [  0,   1,   0,   1,  -1,  -1,  -1],
        [  0,   1,   0,   2,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [  2,   0,   0,   4,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,   1,   5, 300]],

       [[ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1],
        [  3,   1,   5,   4,  -1,  -1,  -1],
        [ -1,  -1,  -1,  -1,  -1,  -1,  -1]]])

## Backtracking with footpaths

When backtracking, with footpaths, we first look through the footpaths to backtrack to the departure stop for the walk, and then use the departure stop of the walk as an arrival stop for a transport.

But with footpaths added, it is possible to reach a stop C from stop A by:
- first taking a transport to a stop B
- walking from stop B to stop C.

Therefore, we need to keep track of all the footpaths taken at step i that ameliorated arrival times at the target stop.



In [61]:
backtrack_journey(k_last, p_t, journey)

******************JOURNEY IN 1 TRIPS******************
raw representation of the journey in 1 trips: [array([ 2,  0,  0,  4, -1, -1, -1])]
At stop 0, take route 2 leaving at time 2020-05-11T08:20 
...
 and exit at stop 4 at time 2020-05-11T09:20
******************JOURNEY IN 2 TRIPS******************
raw representation of the journey in 2 trips: [[array([ 0,  1,  0,  1, -1, -1, -1]), array([ -1,  -1,  -1,  -1,   1,   5, 300])], array([ 3,  1,  5,  4, -1, -1, -1])]
[array([ 0,  1,  0,  1, -1, -1, -1]), array([ -1,  -1,  -1,  -1,   1,   5, 300])]
At stop 0, take route 0 leaving at time 2020-05-11T08:10 
...
... exit at stop 1 at time 2020-05-11T08:35... 
and walk for 5.0 minutes from stop 1 to stop 5.
At stop 5, take route 3 leaving at time 2020-05-11T08:45 
...
 and exit at stop 4 at time 2020-05-11T09:05


True

## Trying to run the standard RAPTOR on real size data
### Loading real sized data

In [62]:
import pickle
# step 1 convert the data from string to numpy series
routes_real = pickle.load( open( "../data/routes_array2.pkl", "rb" ) )
print(routes_real)
print('We find {} routes in the data'.format(len(routes_real)))

[[1 11 0 0]
 [1 11 11 11]
 [1 11 22 22]
 ...
 [1 6 237432 245713]
 [1 13 237438 245719]
 [3 2 237451 245732]]
We find 16210 routes in the data


In [65]:
stops_real = pickle.load(open( "../data/stops_array.pkl", "rb" ) )
print(stops_real)
print('We find {} stops in the data'.format(len(stops_real)))

[[0 None]
 [4 None]
 [7 None]
 ...
 [7841 None]
 [7844 None]
 [7847 None]]
We find 1407 stops in the data


In [66]:
stopTimes_real = pickle.load(open( "../data/stop_times_array1.pkl", "rb" ) )
print(stopTimes_real)
print('We find {} arrival/departure times for stops in the data'.format(len(stopTimes_real)))

[[                          'NaT' '2020-05-21T16:53:00.000000000']
 ['2020-05-21T16:55:00.000000000' '2020-05-21T16:55:00.000000000']
 ['2020-05-21T16:57:00.000000000' '2020-05-21T16:57:00.000000000']
 ...
 ['2020-05-21T15:10:00.000000000'                           'NaT']
 [                          'NaT' '2020-05-21T16:45:00.000000000']
 ['2020-05-21T17:05:00.000000000'                           'NaT']]
We find 245738 arrival/departure times for stops in the data


In [67]:
transfer_real = pickle.load(open( "../data/transfer_array.pkl", "rb" ) )
print(transfer_real)
print('We find {} footpaths (bidirectional) in the data'.format(len(transfer_real)))

[[1166  146]
 [1270  360]
 [   2    8]
 ...
 [ 108  371]
 [ 102  439]
 [1739  519]]
We find 12564 footpaths (bidirectional) in the data


In [68]:
stopRoutes_real = pickle.load(open( "../data/stop_routes_array.pkl", "rb" ) )
# The route index alone was not selected:
#stopRoutes_real = stopRoutes_real[:, 1]
print(stopRoutes_real)
print('We find {} (r, p) route stops combinations in the data'.format(len(stopRoutes_real)))
#print('We find {} unique routes desserving stops'.format(len(np.unique(stopRoutes_real))))

[  0   1  88 ... 736 735 736]
We find 8000 (r, p) route stops combinations in the data


In [72]:
routeStops_real = pickle.load(open( "../data/route_stops_array.pkl", "rb" ) )
print(routeStops_real)
print('We find {} route, stops combinations'.format(len(routeStops_real)))
print('We find {} unique stops desserving routes'.format(len(np.unique(routeStops_real))))

[  0   1   2 ... 759 554 493]
We find 7849 route, stops combinations
We find 1407 unique stops desserving routes


In [71]:
p_s_real = 10 # start stop = A
p_t = 4 # target stop = E
tau_0 = np.datetime64('2020-05-21T12:00:00') # departure time 08:05
k_max = 10 # we set a maximum number of transports to pre-allocate memory for the numpy array tau_i

tau, tau_star, k, journey = raptor_standard(p_s, p_t, tau_0, 
                                            routes = routes_real, routeStops = routeStops_real, stopTimes = stopTimes_real, stopRoutes = stopRoutes_real, stops = stops_real)


******************************STARTING round k=1******************************
Marked stops at the start of the round: [0]
Route considered for the queue: (0, 0)
Route considered for the queue: (1, 0)
Route considered for the queue: (88, 0)
Route considered for the queue: (89, 0)
Queue before traversing each route: [(0, 0), (1, 0), (88, 0), (89, 0)]

****TRAVERSING ROUTE r=0 from stop p=0****
p_i: 0

----scanning departure times for route r=0 at stop p_i=0----
Earliest arrival time at previous step: tau[k-1][p_i]: 2020-05-21T12:00
Departure time considered: t_r_dep: 2020-05-21T16:53:00.000000000

!!!!Hopped on route r=0, trip t=0 at stop p_i=0!!!!
p_i: 1
arr_t_p_i: 2020-05-21T16:55:00.000000000

----scanning departure times for route r=0 at stop p_i=1----
Earliest arrival time at previous step: tau[k-1][p_i]: 2100-01-01T00:00
Departure time considered: t_r_dep: 2020-05-21T16:55:00.000000000
p_i: 2
arr_t_p_i: 2020-05-21T16:57:00.000000000

----scanning departure times for route r=0 at 

IndexError: index 0 is out of bounds for axis 0 with size 0

## Code for prototyping and debugging:

In [ ]:
0

In [ ]:
p_s = 0 # start stop = A
p_t = 4 # target stop = E
tau_0 = np.datetime64('2020-05-11T08:05') # departure time 08:05
k_max = 10 # we set a maximum number of transports to pre-allocate memory for the numpy array tau_i

In [ ]:
# initialization
n_stops = len(stops)

# earliest arrival time at each stop for each round.
tau = np.full(shape=(k_max, n_stops), fill_value = np.datetime64('2100-01-01T00:00')) # 2100 instead of infinity # number of stops * max number of transports

# earliest arrival time at each stop, indep. of round
tau_star = np.full(shape=n_stops, fill_value = np.datetime64('2100-01-01T00:00'))

marked = [p_s]
q = []
tau[0, p_s] = tau_0

In [ ]:
np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i)[0][0]

In [ ]:
routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i

In [ ]:
p_i

In [ ]:
t_r_dep = stopTimes[routes[r][3]+\
                     # offset corresponding to stop p_i in route r
                     np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i)[0][0] + \
                     routes[r][1]*t_r][1]

In [ ]:
if np.where(routeStops[routes[1][2]:routes[1][2]+routes[1][1]] == 2) <\
np.where(routeStops[routes[1][2]:routes[1][2]+routes[1][1]] == 3):
    print("hello")

In [ ]:
routeStops[routes[1][2] + np.where(routeStops[routes[1][2]:routes[1][2]+routes[1][1]] == 2)[0][0]:routes[1][2]+routes[1][1]]

In [ ]:
routeStops[routes[1][2] + np.where(routeStops[routes[1][2]:routes[1][2]+routes[1][1]] == 2)[0][0]:6]

In [ ]:
routeStops[routes[1][2]]

In [ ]:
routeStops[np.where(routeStops[routes[1][2]:routes[1][2]+routes[1][1]] == 2)[0][0]]

In [ ]:
if True and \
        True:
    print("hello")

In [ ]:
tau[0][0]

In [ ]:
stopTimes[3][1]

In [ ]:
a = np.arange(1, 10)
a

In [ ]:
a[1:10:2]

In [ ]:
stopTimes[routes[0][3]+\
          # offset corresponding to stop p_i in route r
         np.where(routeStops[routes[0][2]:routes[0][2]+routes[0][1]] == 0)[0][0]:\
         # end of the trips of r
         routes[0][3]+routes[0][0]*routes[0][1]:\
          # we can jump from the number of stops in r to find the next departure of route r at p_i
         routes[0][1]
         ]
# we may more simply loop through all trips, and stop as soon as the departure time is after the arrival time


In [ ]:
stopTimes[routes[0][3]+\
          # offset corresponding to stop p_i in route r
         np.where(routeStops[routes[0][2]:routes[0][2]+routes[0][1]] == 0)[0][0]][1]

In [ ]:
stopTimes[routes[1][3]+\
          # offset corresponding to stop p_i in route r
         np.where(routeStops[routes[1][2]:routes[1][2]+routes[1][1]] == 3)[0][0] + \
         routes[1][1]*1][1]

In [ ]:
# t_r is a trip that belongs to route r. t_r can take value 0 to routes[r][0]-1
t = None
r = 1
tau_k_1 = tau[0][0]
p_i = 3

t_r = 0
while True:
    
    t_r_dep = stopTimes[routes[r][3]+\
         # offset corresponding to stop p_i in route r
         np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i)[0][0] + \
         routes[r][1]*t_r][1]
    
    if t_r_dep > tau_k_1:
        # retrieving the index of the departure time of the trip in stopTimes
        #t = routes[r][3] + t_r * routes[r][1]
        t = t_r
        break
    t_r += 1
    # we could not hop on any trip at this stop
    if t_r == routes[r][0]:
        break
        
print("done")
print(t)

In [ ]:
r = 1
t = 1
p_i = 2
# 1st trip of route + offset for the right trip + offset for the right stop
stopTimes[routes[r][3] + t * routes[r][1] + np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i)]

In [ ]:
d = []
not d

In [ ]:
r = 1
t = 0
p_i = 4
arr_t_p_i = stopTimes[routes[r][3] + \
                          t * routes[r][1] + \
                          np.where(routeStops[routes[r][2]:routes[r][2]+routes[r][1]] == p_i)[0][0]][0]
arr_t_p_i

In [ ]:
np.datetime64('NaT') > np.datetime64('2100-01-01')

In [ ]:
np.datetime64('NaT') < np.datetime64('2100-01-01')